In [111]:
!pip install sdv

In [112]:
#!unrar x  data.rar

In [113]:
import pandas as pd
import numpy as np
import torch
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from hyperopt import fmin, rand, hp, Trials, STATUS_OK
from hyperopt import space_eval
from sdv.single_table import CTGANSynthesizer as CTGAN
from sdv.metadata import SingleTableMetadata
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [114]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [115]:
    dir_datasets = '/kaggle/input/nir02-main/data/'

    # Загрузка реальных датасетов
    real_data_1 = pd.read_csv(dir_datasets+'insurance.csv')
    real_data_2 = pd.read_csv(dir_datasets+'my_classification.csv')
    real_data_3 = pd.read_csv(dir_datasets+'my_regression.csv')
    real_data_4 = pd.read_csv(dir_datasets+'two_moons.csv')
    real_data_5 = pd.read_csv(dir_datasets+'california_housing.csv')
    real_data_6 = pd.read_csv(dir_datasets+'titanic.csv')

    # Словарь датасетов для удобства
    datasets = {
        # 'insurance': {
        #                 "data": real_data_1,
        #                 "task": "regression",
        #                 "target": "expenses",
        #                 "num_columns":
        #                 ["age", "bmi", "children"],
        #                 "cat_columns":
        #                 ["sex", "smoker", "region"]
        #             },
        'my_classification': {
                        "data": real_data_2,
                        "task": "classification",
                        "target": "target",
                        "num_columns":
                        ["feature_0", "feature_1", "feature_2", "feature_3", "feature_4"],
                        "cat_columns":
                        []
                    },
        'my_regression': {
                        "data": real_data_3,
                        "task": "regression",
                        "target": "target",
                        "num_columns":
                        ["feature_0", "feature_1", "feature_2", "feature_3"],
                        "cat_columns":
                        []
                    },

        'two_moons': {
                        "data": real_data_4,
                        "task": "classification",
                        "target": "label",
                        "num_columns":
                        ["x1", "x2"],
                        "cat_columns": 
                        []
                    },
        # 'california_housing': {
        #                 "data": real_data_5,
        #                 "task": "regression",
        #                 "target": "MedHouseVal",
        #                 "num_columns":
        #                 ["MedInc",	"HouseAge",	"AveRooms",	"AveBedrms","Population",	"AveOccup",	"Latitude",	"Longitude"],
        #                 "cat_columns": 
        #                 []
        #             },
        # 'titanic': {
        #                 "data": real_data_6,
        #                 "task": "classification",
        #                 "target": "Survived",
        #                 "num_columns": ["Age", "Fare", "SibSp", "Parch"],
        #                 "cat_columns": ["Pclass", "Sex", "Embarked"]
        #             },
        
    }

    for name, data in datasets.items():
        print(f" Информация о датасете {name}:")
        print(f" Количество строк: {data['data'].shape[0]}")
        print(f" Количество колонок: {data['data'].shape[1]}")
        print(f" Колонки: {list(data['data'].columns)}")
        print(f" Задача: {data['task']}")
        print(f" Целевая переменная: {data['target']}")
        print(f" Числовые признаки: {data['num_columns']}")
        print(f" Категориальные признаки: {data['cat_columns']}")
        print()

 Информация о датасете my_classification:
 Количество строк: 900
 Количество колонок: 6
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'target']
 Задача: classification
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']
 Категориальные признаки: []

 Информация о датасете my_regression:
 Количество строк: 900
 Количество колонок: 5
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'target']
 Задача: regression
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3']
 Категориальные признаки: []

 Информация о датасете two_moons:
 Количество строк: 900
 Количество колонок: 3
 Колонки: ['x1', 'x2', 'label']
 Задача: classification
 Целевая переменная: label
 Числовые признаки: ['x1', 'x2']
 Категориальные признаки: []



In [116]:
def process_data(dataset, num_columns, cat_columns, transformation_num_type='None', transformation_cat_type='None'):

    df_processed = dataset.copy()

    # Обработка числовых признаков
    if transformation_num_type == 'CDF':
        # CDF трансформация: преобразует значения в их эмпирическую функцию распределения
        for col in num_columns:
            # Правильная формула для эмпирической CDF
            df_processed[col] = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

    elif transformation_num_type == 'PLE_CDF':
        # PLE_CDF (Probability Logit Envelope CDF) - более сложная трансформация
        for col in num_columns:
            # Сначала применяем CDF
            cdf_values = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

            # Затем применяем logit трансформацию с небольшим сглаживанием
            # Избегаем 0 и 1 для предотвращения бесконечности в logit
            epsilon = 1e-6
            cdf_values = np.clip(cdf_values, epsilon, 1 - epsilon)

            # Логит трансформация: ln(p/(1-p))
            df_processed[col] = np.log(cdf_values / (1 - cdf_values))

    # Обработка категориальных признаков
    if transformation_cat_type == 'OHE':
        # One Hot Encoding для категориальных признаков
        for col in cat_columns:
            # Создаем dummy переменные с префиксом имени колонки
            dummy_df = pd.get_dummies(df_processed[col], prefix=col, dtype=int)

            # Удаляем исходную категориальную колонку
            df_processed = df_processed.drop(columns=[col])

            # Добавляем новые dummy колонки
            df_processed = pd.concat([df_processed, dummy_df], axis=1)

    return df_processed

In [117]:
ctgan_space = {
    # Learning rates
    'discriminator_lr': hp.qloguniform('discriminator_lr',
                                                 np.log(4e-4), np.log(2.1e-3), 5e-5),
    'generator_lr': hp.qloguniform('generator_lr',
                                             np.log(5e-5), np.log(5e-3), 5e-5),

    # Batch size
    'batch_size': hp.choice('batch_size', [100, 500, 1000]),

    # Embedding dimensions
    'embedding_dim': hp.choice('embedding_dim', [32, 128]),

    # Network architecture
    'generator_dim': hp.choice('generator_dim', [[128, 128, 128], [128, 128, 128, 128]]),
    'discriminator_dim': hp.choice('discriminator_dim', [[256, 256], [256, 256, 256]]),

    # Decay parameters
    'generator_decay': hp.qloguniform('generator_decay',
                                     np.log(1e-6), np.log(6.4e-6), 1e-7),
    'discriminator_decay': hp.qloguniform('discriminator_decay',
                                         np.log(1e-6), np.log(8e-6), 1e-6),

    # Frequency
    'log_frequency': hp.choice('log_frequency', [False, True]),
    'transformation_num_type': hp.choice('transformation_num_type', ['CDF', 'PLE_CDF']),
    'transformation_cat_type': hp.choice('transformation_cat_type', ['OHE'])

}

In [118]:
def evaluate_c2st(real, synthetic):
    real_copy = real.copy()
    synthetic_copy = synthetic.copy()

    # Добавление меток
    real_copy['label'] = 1
    synthetic_copy['label'] = 0

    # Объединение данных
    df = pd.concat([real_copy, synthetic_copy], ignore_index=True)

    # Определение категориальных колонок
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
    categorical_columns = [col for col in categorical_columns if col != 'label']

    # Label Encoding для категориальных признаков
    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))

    # Разделение на признаки и метки
    X = df.drop(columns='label')
    y = df['label']

    # Разделение на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=RANDOM_SEED
    )
    # Обучение XGBoost
    clf = XGBClassifier(
        random_state=RANDOM_SEED
    )
    clf.fit(X_train, y_train)

    # Предсказания
    y_pred_proba = clf.predict_proba(X_test)[:, 1]

    return roc_auc_score(y_test, y_pred_proba)


In [119]:
from sklearn.model_selection import KFold

def generate_and_evaluate(params, dataset_info, n_splits=3):
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    columns_dataset = dataset_info['num_columns'] + dataset_info['cat_columns'] + [dataset_info['target']]
    data = process_data(dataset_info['data'][columns_dataset], dataset_info['num_columns'], dataset_info['cat_columns'],
                        params['transformation_num_type'], params['transformation_cat_type'])
    
    kf = KFold(n_splits=n_splits, shuffle=False)
    c2st_scores = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index].reset_index(drop=True)
        test_data = data.iloc[test_index].reset_index(drop=True)

        metadata = SingleTableMetadata()
        metadata.detect_from_dataframe(data=train_data)

        # Создание и обучение CTGAN с заданными параметрами
        ctgan = CTGAN(
            metadata=metadata,
            discriminator_lr=params['discriminator_lr'],
            generator_lr=params['generator_lr'],
            batch_size=params['batch_size'],
            embedding_dim=params['embedding_dim'],
            generator_dim=params['generator_dim'],
            generator_decay=params['generator_decay'],
            discriminator_decay=params['discriminator_decay'],
            log_frequency=params['log_frequency'],
        )

        ctgan.fit(train_data)

        synthetic_data = ctgan.sample(len(test_data))

        score = evaluate_c2st(test_data, synthetic_data)
        c2st_scores.append(score)

    return np.mean(c2st_scores)

In [120]:
def optimize_dataset(dataset_name, dataset_info, max_evals=30):
    print(f"Оптимизация для датасета: {dataset_name}")

    # Создаем объект для хранения истории поиска
    trials = Trials()

    # Определяем целевую функцию для оптимизации
    def objective(params):
        print("="*50)
        c2st_score = generate_and_evaluate(params, dataset_info)

        print(c2st_score)
        print(params)
        print("="*50)
        # Возвращаем словарь в формате, который ожидает hyperopt
        return {
            'loss': c2st_score,
            'status': STATUS_OK,
            'c2st_score': c2st_score
        }


    # Запускаем оптимизацию с помощью TPE алгоритма
    rng = np.random.default_rng(RANDOM_SEED)
    best = fmin(
        fn=objective,
        space=ctgan_space,
        algo=rand.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=rng
    )

    # Автоматическое декодирование параметров
    best_params = space_eval(ctgan_space, best)

    # Находим лучший результат
    best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
    best_c2st = best_trial['result'].get('c2st_score', None)

    results = {
        'best_params': best_params,
        'best_c2st': best_c2st,
        'c2st_diff_from_optimal': best_trial['result']['loss']
    }

    print(f"Лучший C2ST для {dataset_name}: {best_c2st}")
    print(f"Лучшие параметры: {best_params}")
    print('-' * 50)

    return results

In [121]:
# Словарь для хранения результатов
all_results = {}

# Указываем количество итераций для каждого датасета
max_evals = 50

# Запускаем оптимизацию для каждого датасета
for dataset_name in datasets:
    data_info = datasets[dataset_name]
    all_results[dataset_name] = optimize_dataset(dataset_name, data_info, max_evals)

Оптимизация для датасета: my_classification
0.7026748971193415                                    
{'batch_size': 100, 'discriminator_decay': 4e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.0014500000000000001, 'embedding_dim': 128, 'generator_decay': 3.3999999999999996e-06, 'generator_dim': (128, 128, 128), 'generator_lr': 0.0008500000000000001, 'log_frequency': True, 'transformation_cat_type': 'OHE', 'transformation_num_type': 'CDF'}
0.7165432098765433                                                                 
{'batch_size': 100, 'discriminator_decay': 6e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.001, 'embedding_dim': 128, 'generator_decay': 1.1e-06, 'generator_dim': (128, 128, 128, 128), 'generator_lr': 0.0008, 'log_frequency': False, 'transformation_cat_type': 'OHE', 'transformation_num_type': 'PLE_CDF'}
0.7716049382716049                                                                 
{'batch_size': 1000, 'discriminator_decay': 6e-06

In [122]:
for dataset_name in datasets:
    print(dataset_name, all_results[dataset_name]['best_c2st'])

my_classification 0.6865432098765432
my_regression 0.7595473251028807
two_moons 0.7425102880658437
